In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep # control the crawl rate to avoid hammering the servers with too many requests
from random import randint
import re # regular expression

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities # dealing with pop-up permissions
from selenium.webdriver.firefox.options import Options

In [ ]:
# what i need to get from the reviews
# 1) stars (0 - 5) ## "data-comp="StarRating" <div aria-label="4 stars">
# 2) reviewer id ## <a href="users/Tarah84" ...> under <strong data-at="nickname">____</strong>
# 3) reviewer features ## two lines under "nickname"
# 4) purchased shade ## line after headline of the review ("<h3 class>")

reviewer_id = []
rating = []
review_subject = []
reviewer_feature = []
purchased_shade = []

cols = {
    "reviewer_id": reviewer_id, "rating": rating, "review_subject": review_subject,
    "reviewer_feature": reviewer_feature, "purchased_shade": purchased_shade
    }

fenty_dat = pd.DataFrame(cols)


In [ ]:
geoBlocked = webdriver.FirefoxOptions()
geoBlocked.set_preference("geo.prompt.testing", True)
geoBlocked.set_preference("geo.prompt.testing.allow", False)
driver = webdriver.Firefox(options=geoBlocked)
driver.get("https://www.sephora.com/product/pro-filtr-soft-matte-longwear-foundation-P87985432?skuId=2164697")
sleep(3)
driver.execute_script("window.scrollTo(0, 3000)") 
sleep(2)
driver.execute_script("window.scrollTo(0, 4500)") 

In [ ]:
# preparing to click
# driver.find_elements_by_css_selector('div[data-comp="Pagination Flex StyledComponent BaseComponent "')

In [ ]:
list_of_reviews = driver.find_elements_by_css_selector('div[data-comp="Review StyledComponent BaseComponent "]')

list_of_reviews[0].find_elements_by_css_selector('strong[data-at="nickname"]')[0].text


def append_new_data(data = fenty_dat, list_of_reviews = list_of_reviews):
    shade = ""
    for i in list_of_reviews:
        if i.find_elements_by_css_selector('img[src*="https://www.sephora.com/productimages/"]+span') != []:
            shade = i.find_elements_by_css_selector('img[src*="https://www.sephora.com/productimages/"]+span')[0].text
        else:
            shade = None

        new_col = {
            "reviewer_id": i.find_elements_by_css_selector('strong[data-at="nickname"]')[0].text,
            "rating": int(i.find_elements_by_css_selector('div[data-comp="StarRating "]')[0].get_attribute("aria-label")[0]),
            "review_subject": 
            "reviewer_feature": i.find_elements_by_css_selector('strong[data-at="nickname"]+span')[0].text,
            "purchased_shade": shade
        }
        # print(new_col)
        data = data.append(new_col,ignore_index=True)
    return data

In [ ]:

driver.find_elements_by_css_selector('ul[data-comp="Pagination Flex StyledComponent BaseComponent "]:nth-child(8) button[data-at="pagination_page"]')[-1].text

In [ ]:
driver.find_element_by_css_selector('button[aria-label="Next"]').click()

In [ ]:

driver.find_element_by_css_selector('button[aria-label="Next"][disabled=""]')

In [ ]:
fenty_dat = append_new_data()

In [ ]:
fenty_dat